# Ternary Classifier

Create a classifier model using the VGG16 architecture and train it to classify between three classes. Originally used with the arXiv images dataset and the classes of diagram, sensor, and unsure.

Draws upon code from Florian Offert: https://github.com/zentralwerkstatt/explain.ipynb/blob/master/train.ipynb

In [10]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np

import os
import sys
import glob
import random
from IPython.display import clear_output, display
import PIL.Image

# from keras.applications.inception_v3 import InceptionV3, preprocess_input
# from keras.applications.vgg16 import VGG16, preprocess_input
# from keras.applications.densenet import DenseNet121 as DenseNet, preprocess_input
from keras.applications.inception_resnet_v2 import InceptionResNetV2 as ResNet, preprocess_input

from keras.models import Model, load_model
from keras.layers import Dense, GlobalAveragePooling2D, AveragePooling2D, Dropout, Flatten, Conv2D, Activation
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, img_to_array # load_img
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt

In [11]:
# floating point 16

import keras.backend as K

# dtype='float16'
dtype='float32'
K.set_floatx(dtype)

# default is 1e-7 which is too small for float16.  Without adjusting the epsilon, we will get NaN predictions because of divide by zero problems
K.set_epsilon(1e-4) 

In [12]:
# this seems to help with some GPU memory issues

import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [13]:
# train_dir = '/home/rte/data/images/random/seq/classes/train/'
# val_dir = '/home/rte/data/images/random/seq/classes/val'

train_dir = '/home/rte/data/images/classification/labelled/train/'
val_dir = '/home/rte/data/images/classification/labelled/val'

classes = 3
batch_size = 32
epochs = 2000
save_iter = 50
# save_iter = 25
finetune = False
weights = 'imagenet'
size = 224 # VGG16
# size = 299 # V3
freeze = 164 # V3, up to and including mixed5
freeze = 20 # VGG16, up to and including 
# cpath = 'diagram-sensor-unsure_vgg16-{epoch:02d}.hdf5'
cpath = 'diagram-sensor-unsure_dnet-{epoch:02d}.hdf5'
checkpoints_path = "checkpoints/"
init_epoch = 25

In [14]:
def add_new_last_layer_v3(base_model, classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(classes, activation='softmax', name='Predictions')(x)
    model = Model(input=base_model.input, output=predictions)
    return model

def setup_to_finetune(model):
    for layer in model.layers[:freeze]:
        layer.trainable = False
    for layer in model.layers[freeze:]:
        layer.trainable = True
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    
def setup_to_train(model):
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(size, size),
    batch_size=batch_size,
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(size, size),
    batch_size=batch_size,
)

Found 7799 images belonging to 3 classes.
Found 1949 images belonging to 3 classes.


In [16]:
nb_train_samples = 7799
nb_validation_samples = 1949

Train using stochastic gradient descent. Results were mixed using Adam.

In [17]:
# We need to define the input shape, see 
# https://stackoverflow.com/questions/49043955/shape-of-input-to-flatten-is-not-fully-defined-got-none-none-64
# base_model = VGG16(weights=None, input_shape=(size, size, 3), include_top=False)
# base_model = InceptionV3(weights=weights, input_shape=(size, size, 3), include_top=False)
# base_model = DenseNet(weights=None, input_shape=(size,size,3), include_top=False)
base_model = ResNet(weights=None, input_shape=(size,size,3), include_top=False)
base_model.summary()

# freeze layers
# for layer in base_model.layers:
#     layer.trainable = False

layer_dict = dict([(layer.name, n) for n, layer in enumerate(base_model.layers)])
model = add_new_last_layer_v3(base_model, classes)
# model.summary()

# if finetune: setup_to_finetune(model)
# else: setup_to_train(model)

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

W0309 14:54:37.745035 139865495971648 deprecation_wrapper.py:119] From /home/rte/.local/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_204 (Conv2D)             (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_204 (BatchN (None, 111, 111, 32) 96          conv2d_204[0][0]                 
__________________________________________________________________________________________________
activation_204 (Activation)     (None, 111, 111, 32) 0           batch_normalization_204[0][0]    
__________________________________________________________________________________________________
conv2d_205

activation_263 (Activation)     (None, 25, 25, 64)   0           batch_normalization_263[0][0]    
__________________________________________________________________________________________________
block35_8_mixed (Concatenate)   (None, 25, 25, 128)  0           activation_258[0][0]             
                                                                 activation_260[0][0]             
                                                                 activation_263[0][0]             
__________________________________________________________________________________________________
block35_8_conv (Conv2D)         (None, 25, 25, 320)  41280       block35_8_mixed[0][0]            
__________________________________________________________________________________________________
block35_8 (Lambda)              (None, 25, 25, 320)  0           block35_7_ac[0][0]               
                                                                 block35_8_conv[0][0]             
__________

In [18]:
class_weights = {0: 1,
                1: 18,
                2: 14}

In [ ]:
# callbacks = [
#     EpochCheckpoint(checkpoints_path, every=25, startAt=),
#     CSVLogger("log.csv", separator=',', append=False)
# ]

In [20]:
history = model.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights,
    steps_per_epoch=nb_train_samples // batch_size,
    validation_steps=nb_validation_samples // batch_size,
    initial_epoch=init_epoch,
    callbacks=[ModelCheckpoint(cpath, period=save_iter)])

# keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

Epoch 1/2000
 10/243 [>.............................] - ETA: 1:30 - loss: 1.3705 - acc: 0.8656

KeyboardInterrupt: 

In [ ]:
# plot training metrics

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model = load_model('diagram-sensor-unsure_vgg16-500.hdf5')

In [ ]:
model = load_model('ternary_20190911_9748x/diagram-sensor-unsure_vgg16-2000.hdf5')

In [ ]:
# was this trying to convert for visualisation?

# from tensorflow.python.framework import graph_util
# graph_util.convert_variables_to_constants(...)

In [ ]:
all_val_images = []

list_of_labels = os.listdir(val_dir)
for label in list_of_labels:
    current_label_dir_path = os.path.join(val_dir, label)
    list_of_images = os.listdir(current_label_dir_path)
    for im in list_of_images:
        current_image_path = os.path.join(current_label_dir_path, im)
#         image = Image.open(current_image_path) # use the function which you want.
#         print(current_image_path)
        all_val_images.append(current_image_path)
print(len(all_val_images))

In [ ]:
def load_image(path):
    img = image.load_img(path, target_size=model.input_shape[1:3])
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

Iteratively go through validation images, get the prediction and 'ground truth', save image

In [ ]:
if not os.path.exists("predictions"): 
    os.mkdir("predictions")

classes = ["diagram", "sensor", "unsure"]
results = ["true", "false"]

for c in classes:
    for r in results:
        newdir = os.path.join("predictions/", c + "_" + r)
        print(newdir)
        if not os.path.exists(newdir):
            os.mkdir(newdir)

In [ ]:
# img, x = load_image("/home/rte/data/images/random/seq/classes/val/unsure/362696.jpg")

classes = ["diagram", "sensor", "unsure"]

# iterate over all validation samples
for i in range(0, nb_validation_samples):

    # get a random filepath
#     r = random.randint(0, len(all_val_images))
    
    # not random anymore
    r = i
    print(r)
    random_file = all_val_images[r]
    print(random_file)

    img, x = load_image(random_file)
    # print("shape of x: ", x.shape)
    # print("data type: ", x.dtype)

    # forward the image through the network
    predictions = model.predict(x)
    # print(predictions)
    print("diagram: %0.3f | sensor: %0.3f | unsure %0.3f" % \
          (predictions[0][0], predictions[0][1], predictions[0][2]))

    # draw the figure
    fig = plt.figure()
    plt.imshow(img)
    plt.axis('off')
    # grab the class from the filepath
    c = random_file.rsplit("/")[-2]
    label = c

    # add the 'ground truth' label and prediction as the title
    plt.title("class: " + label + "\n" \
              + "diagram: %0.3f | sensor: %0.3f | unsure %0.3f" % \
          (predictions[0][0], predictions[0][1], predictions[0][2]), loc='left')

    # save image
    # which index is the result
    ind = np.argmax(predictions)
    pred = classes[ind]
    # compare prediction and label
    # 
    result = True if classes.index(label) == ind else False
    
    savefolder = os.path.join("predictions/", pred + "_" + "true" if result else pred + "_" + "false")
    
    savepath = os.path.join(savefolder, "prediction_e500_" + random_file.rsplit("/")[-1])
    print(savepath)
    fig.savefig(savepath, dpi=150)

In [ ]:
classes = ["diagram", "sensor", "unsure"]

print(predictions)
ind = np.argmax(predictions)
print(ind)
pred = classes[ind]
print(pred)
# compare prediction and label
# 
result = True if classes.index(label) == ind else False
print(result)

In [ ]:
print(classes)

Get a folder of unseen images, run the classifier over them, and save the images with the classification

In [ ]:
unseen_images = []
unseen_dir = "/home/rte/data/images/random/seq/0-100k/unseen/"

list_of_images = os.listdir(unseen_dir)
for im in list_of_images:
    current_image_path = os.path.join(current_label_dir_path, im)
    unseen_images.append(current_image_path)
nb_unseen_samples = len(unseen_images)
print(len(unseen_images))

In [ ]:
if not os.path.exists("predictions_unseen"): 
    os.mkdir("predictions_unseen")

classes = ["diagram", "sensor", "unsure"]

for c in classes:
    newdir = os.path.join("predictions_unseen/", c)
    print(newdir)
    if not os.path.exists(newdir):
        os.mkdir(newdir)

In [ ]:
# get unseen images predictions

classes = ["diagram", "sensor", "unsure"]

# iterate over all unseen samples
# nb_unseen_samples
for r in range(0, nb_unseen_samples):

    print(r)
    random_file = all_val_images[r]
    print(random_file)

    img, x = load_image(random_file)
    # print("shape of x: ", x.shape)
    # print("data type: ", x.dtype)

    # forward the image through the network
    predictions = model.predict(x)
    # print(predictions)
    print("diagram: %0.3f | sensor: %0.3f | unsure %0.3f" % \
          (predictions[0][0], predictions[0][1], predictions[0][2]))

    # draw the figure
    fig = plt.figure()
    plt.imshow(img)
    plt.axis('off')
    # grab the class from the filepath
#     c = random_file.rsplit("/")[-2]
#     label = c

    # add the prediction as the title
    plt.title("diagram: %0.3f | sensor: %0.3f | unsure %0.3f" % \
          (predictions[0][0], predictions[0][1], predictions[0][2]), loc='left')

    # save image
    # which index is the result
    ind = np.argmax(predictions)
    pred = classes[ind]
    # compare prediction and label
    # 
#     result = True if classes.index(label) == ind else False
    
    savefolder = os.path.join("predictions_unseen/", pred)
    
    savepath = os.path.join(savefolder, "prediction_" + random_file.rsplit("/")[-1])
    print(savepath)
    fig.savefig(savepath, dpi=150)

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')